<a href="https://colab.research.google.com/github/Contsummer/SVM/blob/main/mau1ki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install faker
import pandas as pd
import numpy as np
from faker import Faker
import random

fake = Faker('vi_VN')

def generate_student_data(n_students=1000):
    data = []
    for _ in range(n_students):
        name = fake.name()
        mssv = f"2020{random.randint(10000, 99999)}"
        performance = random.random()
        health = random.randint(1, 3)
        location = random.random()

        scores = {}
        credits_per_subject = []
        for sub in range(1, 6):
            subject_credits = random.randint(2, 4)
            initial_score = np.clip(np.random.normal(performance * 10, 2), 0, 10)
            final_score = initial_score if initial_score >= 5 else np.clip(np.random.normal(initial_score + 1, 1), 0, 10)
            scaled_score = round(final_score * 0.4, 2)  # Thang 4, làm tròn 2 chữ số
            scores[f"mon{sub}"] = scaled_score
            credits_per_subject.append(subject_credits)

        # Tính tổng số môn và tín chỉ
        total_subjects = len(scores)
        passed_subjects = sum(1 for score in [v / 0.4 for v in scores.values()] if score >= 5)
        failed_subjects = total_subjects - passed_subjects

        # Tính số tín chỉ tích lũy và rớt
        credits_accumulated = sum(credits_per_subject[i] for i in range(total_subjects) if [v / 0.4 for v in scores.values()][i] >= 5)
        credits_failed = min(sum(credits_per_subject[i] for i in range(total_subjects) if [v / 0.4 for v in scores.values()][i] < 5), 20)  # Giới hạn tối đa 20

        while credits_accumulated <= credits_failed:
            credits_accumulated += random.randint(5, 10)

        retake_count = failed_subjects

        if location <= 0.25:
            distance = "gần"
        elif location <= 0.5:
            distance = "trung bình"
        elif location <= 0.75:
            distance = "xa"
        else:
            distance = "rất xa"

        residence_probs = {"ktx": 0.5 if distance in ["xa", "rất xa"] else 0.2,
                           "ở nhà": 0.7 if distance == "gần" else 0.3,
                           "ở trọ": 0.4}
        residence = random.choices(["ktx", "ở nhà", "ở trọ"],
                                   weights=[residence_probs[r] for r in ["ktx", "ở nhà", "ở trọ"]], k=1)[0]

        if residence == "ktx":
            distance = "gần"

        dropout_prob = 1 / (1 + np.exp(5 * performance + 0.5 * health -
                                       0.02 * credits_failed - 0.1 * retake_count + 0.01 * credits_accumulated - 2))
        dropout = 1 if random.random() < dropout_prob else 0

        record = {"Tên": name, "MSSV": mssv, "Sức khỏe": health, "Khoảng cách": distance,
                  "Cư trú": residence, "Số tín chỉ tích lũy": credits_accumulated,
                  "Số tín chỉ rớt": credits_failed, "Số lần học lại": retake_count}
        record.update(scores)
        record["Nghỉ học"] = dropout
        data.append(record)

    return pd.DataFrame(data)

if __name__ == "__main__":
    df = generate_student_data(10000)
    df.to_excel("student_data.xlsx", index=False, engine='openpyxl')
    print("Đã tạo và lưu 100 bản ghi vào 'student_data.xlsx'.")

Đã tạo và lưu 100 bản ghi vào 'student_data.xlsx'.


In [11]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

df = pd.read_excel("student_data.xlsx", engine='openpyxl')

numeric_columns = ['Số tín chỉ tích lũy', 'Số tín chỉ rớt', 'Số lần học lại',
                   'mon1', 'mon2', 'mon3', 'mon4', 'mon5']
categorical_columns = ['Sức khỏe', 'Khoảng cách', 'Cư trú', 'Nghỉ học']

scaler = MinMaxScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

df.to_excel("student_data_normalized.xlsx", index=False, engine='openpyxl')
print("Đã chuẩn hóa dữ liệu và lưu vào 'student_data_normalized.xlsx'.")

Đã chuẩn hóa dữ liệu và lưu vào 'student_data_normalized.xlsx'.


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import joblib

df = pd.read_excel("student_data_normalized.xlsx", engine='openpyxl')

X = df.drop(['Tên', 'MSSV', 'Nghỉ học'], axis=1)
y = df['Nghỉ học']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

model = SVC(kernel='rbf', class_weight='balanced', C=0.1, gamma='scale', probability=True, random_state=42)
model.fit(X_train_resampled, y_train_resampled)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Độ chính xác của mô hình SVM (RBF kernel): {accuracy * 100:.2f}%")
print("\nBáo cáo phân loại chi tiết:")
print(classification_report(y_test, y_pred, target_names=['Không', 'Có']))  # 0: Không nghỉ học, 1: Có nghỉ học



Độ chính xác của mô hình SVM (RBF kernel): 75.33%

Báo cáo phân loại chi tiết:
              precision    recall  f1-score   support

       Không       0.90      0.73      0.81      2096
          Có       0.56      0.80      0.66       904

    accuracy                           0.75      3000
   macro avg       0.73      0.77      0.73      3000
weighted avg       0.80      0.75      0.76      3000



In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import joblib
import numpy as np

# Đọc dữ liệu từ file Excel đã chuẩn hóa
df = pd.read_excel("student_data_normalized.xlsx", engine='openpyxl')

X = df.drop(['Tên', 'MSSV', 'Nghỉ học'], axis=1)
y = df['Nghỉ học']  # Đã mã hóa thành 0 và 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Áp dụng SMOTE để xử lý mất cân bằng dữ liệu
smote = SMOTE(sampling_strategy=0.8, random_state=42)  # Tăng số mẫu lớp thiểu số
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

model = SVC(kernel='rbf',
            C=0.5,  # Tăng C để nhạy hơn
            gamma="scale",  # Giảm gamma để ranh giới mượt hơn
            class_weight={0: 1, 1: 5},  # Tăng trọng số cho lớp thiểu số
            probability=True,
            random_state=42)
model.fit(X_train_resampled, y_train_resampled)

y_pred_proba = model.predict_proba(X_test)[:, 1]  # Xác suất lớp 1 ("Nghỉ học")

threshold = 0.5
y_pred = (y_pred_proba >= threshold).astype(int)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f"Độ chính xác của mô hình SVM (RBF kernel) với ngưỡng {threshold}: {accuracy * 100:.2f}%")
print("\nBáo cáo phân loại chi tiết:")
print(classification_report(y_test, y_pred, target_names=['Không', 'Có']))



Độ chính xác của mô hình SVM (RBF kernel) với ngưỡng 0.5: 70.53%

Báo cáo phân loại chi tiết:
              precision    recall  f1-score   support

       Không       0.93      0.62      0.75      2096
          Có       0.51      0.90      0.65       904

    accuracy                           0.71      3000
   macro avg       0.72      0.76      0.70      3000
weighted avg       0.80      0.71      0.72      3000

